# Dados

In [5]:
import os
from pathlib import Path
import pandas as pd
import numpy as np
import cv2
import shutil
from module import data_read, setup
from torch.utils.data import ConcatDataset
from module.LightningModule import ClassificationModule
from module.setup import *
from module.data_read import *

torch.manual_seed(random_state)

DDR_DIR = Path.cwd() / 'DDR-dataset/DR_grading'
IDRID_DIR = Path.cwd() / 'IDRID-Classificacao'
#FGADR_DIR = Path.cwd() / 'FGADR Dataset/Seg-set's

## Binary

In [16]:
# Leitura DDR
train_ddr_dataset = DDRDataset(DDR_DIR/'train',
                           DDR_DIR/'train.txt',           
                           convert_to_binary=True)
valid_ddr_dataset = DDRDataset(DDR_DIR/'valid',
                           DDR_DIR/'valid.txt',         
                           convert_to_binary=True)
test_ddr_dataset = DDRDataset(DDR_DIR/'test',
                          DDR_DIR/'test.txt',          
                          convert_to_binary=True)

ddr_dataset = ConcatDataset([train_ddr_dataset, valid_ddr_dataset, test_ddr_dataset])



# Leitura IDRID
train_idrid_dataset = IDRIDDataset(IDRID_DIR/'1. Original Images/a. Training Set',
                                   IDRID_DIR/'2. Groundtruths/a. IDRiD_Disease Grading_Training Labels.csv',
                                   data_transforms['test'],
                                   convert_to_binary=True)
test_idrid_dataset = IDRIDDataset(IDRID_DIR/'1. Original Images/b. Testing Set',
                                  IDRID_DIR/'2. Groundtruths/b. IDRiD_Disease Grading_Testing Labels.csv',                   
                                  data_transforms['test'],
                                  convert_to_binary=True)
idrid_dataset = ConcatDataset([train_idrid_dataset, test_idrid_dataset])

#### MobileNet v2 Instanciation

In [21]:
from torchvision import models
from torch import nn

# Instanciação Mobilenetv2
mobi_best_weights = models.MobileNet_V2_Weights.DEFAULT
mobi_model = models.mobilenet_v2(weights=mobi_best_weights)
for param in mobi_model.parameters():
    param.requires_grad = False

num_ftrs = mobi_model.classifier[-1].in_features

mobi_model.classifier[-1] = nn.Linear(num_ftrs, num_classes)

In [ ]:
mobi_model

#### Execution

In [17]:
from lightning.pytorch.loggers import TensorBoardLogger
from sklearn.model_selection import KFold

kfold = KFold(n_splits=k_folds, shuffle=True, random_state=random_state)

for fold, (train_ids, test_ids) in enumerate(kfold.split(ddr_dataset)):
    print(f'FOLD {fold}')
    print('--------------------------------')
    # Divide os dados com base nos atuais indices do K-fold
    train_dataset = torch.utils.data.Subset(ddr_dataset, train_ids)
    test_dataset = torch.utils.data.Subset(ddr_dataset, test_ids)


    # Definição dos dataloaders
    trainloader = torch.utils.data.DataLoader(
                      train_dataset,
                      batch_size=batch_size,
                      shuffle=True,
                      num_workers=5)
    testloader = torch.utils.data.DataLoader(
                      test_dataset,
                      batch_size=batch_size,
                      shuffle=False,
                      num_workers=5)
    idridloader = torch.utils.data.DataLoader(
                      idrid_dataset,
                      batch_size=batch_size,
                      num_workers=5)

    # DEBUG:
    # trainer = L.Trainer(fast_dev_run=4) # a execução do trainer se limitará a 2 batches
    # trainer = L.Trainer(limit_train_batches=30, max_epochs=1) # usar apenas 30% dos dados de cada lote de treino
    trainer = L.Trainer(default_root_dir='checkpoints kfold/', accelerator='gpu', max_epochs=num_epochs)


    model = mobi_model
    # treino
    model = ClassificationModule(model, loss_function, optim.Adam)
    trainer.fit(model=model, train_dataloaders=trainloader)
    # teste
    print(f'\n\n TEST FOLD {fold}\n\n')    
    trainer.test(model=model, dataloaders=testloader) # test na fold de test 
    print('\n\n TESTE IDRID \n\n')    
    trainer.test(model=model, dataloaders=idridloader) # test no idrid
    

NameError: name 'ddr_dataset' is not defined